In [5]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Utah 2018

## VEST Documentation

### Data Sources

Election results from OpenElections.

Shapefile from the Utah Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/).


### Election Processing

The shapefile from the UGRC is of subprecincts, breaking down precincts in cases of district splits. In some cases, results are reported at the subprecinct level, in most cases, they weren't, so merging was done where necessary.


### Candidates

G16PRERTRU - Donald J. Trump (Republican Party) 
G16PREDCLI - Hillary Rodham Clinton (Democratic Party)
G16PRELJOH - Gary Johnson (Libertarian Party)
G16PREISTE - Jill Stein (Unaffiliated)
G16PREIMCM - Evan McMullen (Unaffiliated)
G16PREOOTH - Other candidates on ballot

G16USSRLEE - Mike Lee (Republican Party)
G16USSDSNO - Misty K. Snow (Democratic Party)
G16USSAFON - Stoney Fonua (Independent American Party)
G16USSIBAR - Bill Barron (Unaffiliated)

G16GOVRHER - Gary R. Herbert (Republican Party)
G16GOVDWEI - Mike Weinholtz (Democratic Party)
G16GOVLKAM - Brian E. Kamerath (Libertarian Party)
G16GOVASCH - Superdell Schanze (Independent American Party)

G16ATGRREY - Sean D. Reyes (Republican Party)
G16ATGDHAR - Jon V. Harper (Democratic Party)
G16ATGLMCC - W. Andrew McCullogh (Libertarian Party)
G16ATGAISB - Michael W. IsBell (Independent American Party)

G16AUDRDOU - John Rougall (Republican Party)
G16AUDDMIT - Mike Mitchell (Democratic Party)
G16AUDAGRE - Jared Green (Independent American Party)

G16TRERDAM - David Damschen (Republican Party)
G16TREDHAN - Neil A. Hansen (Democratic Party)
G16TRECPRO - Richard Proctor (Constitution Party)


In [6]:
#Load UT election results at the precinct level
ut_election_results = pd.read_csv('raw-from-source/20161108__ut__general__precinct.csv')

#Load the final VEST file
ut_vest = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")

#Load the UT 2016 VEST file
ut_vest_2016 = gp.read_file("raw-from-source/ut_2016/ut_2016.shp")

#Load UT election results at the county level
ut_vest_county = pd.read_csv('raw-from-source/20161108__ut__general__county.csv')

#Load in UGRS shapefile
ut_geo = gp.read_file('raw-from-source/Utah_Vista_Ballot_Areas/VistaBallotAreas.shp')

In [7]:
display(ut_vest)

,CountyID,PrecinctID,county,precinct,G16PRERTRU,G16PREDCLI,G16PRELJOH,G16PREISTE,G16PREIMCM,G16PREOOTH,...,G16ATGDHAR,G16ATGLMCC,G16ATGAISB,G16AUDRDOU,G16AUDDMIT,G16AUDAGRE,G16TRERDAM,G16TREDHAN,G16TRECPRO,geometry
0,1,BV01,Beaver,Beaver 1,469,46,6,1,83,3,...,48,18,17,481,69,27,454,80,36,"POLYGON ((358362.560 4238423.840, 358372.350 4..."
1,1,BV02,Beaver,Beaver 2,334,38,4,4,74,4,...,48,25,14,345,72,21,329,90,19,"POLYGON ((355866.750 4240506.080, 356860.230 4..."
2,1,BV03,Beaver,Beaver 3,268,22,3,0,49,2,...,31,6,11,269,44,14,254,52,18,"POLYGON ((358902.400 4261055.000, 358909.700 4..."
3,1,BV04,Beaver,Beaver 4,101,10,3,0,23,0,...,16,9,3,92,23,12,94,19,14,"POLYGON ((367866.500 4262536.100, 367842.500 4..."
4,1,GR01,Beaver,Greenville,76,10,0,2,4,0,...,12,0,0,67,16,0,57,20,5,"POLYGON ((348977.460 4223848.220, 348973.090 4..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2291,7,RO81,Duchesne,RO81,216,9,2,1,18,0,...,10,15,3,216,16,8,198,14,28,"POLYGON ((580836.220 4464060.720, 580837.220 4..."
2292,7,RO82,Duchesne,RO82,75,15,2,0,13,1,...,16,6,0,72,24,4,70,21,10,"POLYGON ((586967.910 4461063.280, 586317.330 4..."
2293,7,RO91,Duchesne,RO91,0,0,0,0,2,1,...,0,0,0,3,0,0,2,0,1,"POLYGON ((583252.560 4462118.640, 583274.150 4..."
2294,7,TA11,Duchesne,TA11,197,12,3,0,17,0,...,17,4,6,175,30,11,173,26,17,"POLYGON ((533169.000 4511919.160, 533169.000 4..."


In [8]:
display(ut_election_results)

,county,precinct,office,district,candidate,party,votes
0,Beaver,Beaver 1,President,NaN,Rocky De La Fuente,NaN,0
1,Beaver,Beaver 2,President,NaN,Rocky De La Fuente,NaN,0
2,Beaver,Beaver 3,President,NaN,Rocky De La Fuente,NaN,0
3,Beaver,Beaver 4,President,NaN,Rocky De La Fuente,NaN,0
4,Beaver,Greenville,President,NaN,Rocky De La Fuente,NaN,0
...,...,...,...,...,...,...,...
83531,Weber,WCW002,State House,29.0,Lee Perry,Republican,510
83532,Weber,WCW002,State House,29.0,Angela Urrea,Democratic,90
83533,Weber,WHV006,State House,29.0,Lee Perry,Republican,234
83534,Weber,WHV006,State House,29.0,Angela Urrea,Democratic,40


In [9]:
print(ut_election_results.candidate.unique())

['Rocky De La Fuente' 'Jill Stein' 'Alyson Kennedy' 'Gary Johnson'
 'Rocky Giordani' 'Donald Trump' 'Darrell Castle' 'Hillary Clinton'
 'Monica Moorehead' 'Evan McMullin' 'Write-In' 'Misty Snow' 'Bill Barron'
 'Stoney Fonua' 'Mike Lee' 'Chris Stewart' 'Charlene Albarran'
 'Paul McCollaum' 'Gary Herbert' 'Dell Schanze' 'Brian Kamerath'
 'Mike Weinholtz' 'Michael Isbell' 'Jon Harper' 'Sean Reyes'
 'W. Andrew McCullough' 'Ralph Okerlund' 'Kirk Pearson' 'Merrill Nelson'
 'Cindee Beard' 'Mike Noel' 'Ty Markham' 'Rob Bishop' 'Craig Bowden'
 'Chadwick Fairbanks' 'Peter Clemens' 'Sherry Phipps' 'Scott Sandall'
 'Lee Perry' 'Angela Urrea' 'Mike Smith' 'Laurence Kotlikoff'
 'Tony Valdivia' 'L.S. Brown' 'Lyle Hillyard' 'Edward Redd' 'Val Potter'
 'David Clark' 'R Curt Webb' 'Stephen Tryon' 'Jason Chaffetz'
 'David Hinkins' 'Heidi Redd' 'Brad King' 'Christine Watkins'
 'Cole Capener' 'Logan Wilde' 'Fuente' 'Giordani' 'Stephen Park'
 'Hoefling' 'Limbaugh' 'Dustin Baird' 'Andrew Basiago' 'Emidio Sol

In [ ]:
race_dict = {
    'Donald Trump': 'G16PRERTRU',
    'Hillary Clinton'
    'Gary Johnson'
    'Jill Stein' 
    'Evan McMullin'
    
    
    'Mike Lee'
    'Misty Snow' 
    'Stoney Fonua'
    'Bill Barron'
    
    'Gary Herbert'
    'Mike Weinholtz'
    'Brian Kamerath'
    'Dell Schanze' 
    
    'Sean Reyes'
    'Jon Harper' 
    'W. Andrew McCullough'
    'Michael Isbell'
    
    
    
    
    
    'Rocky De La Fuente': 'Alyson Kennedy' 
 'Rocky Giordani' 'Darrell Castle' 
 'Monica Moorehead'  'Write-In' 
   'Chris Stewart' 'Charlene Albarran'
 'Paul McCollaum'  
  
  'Ralph Okerlund' 'Kirk Pearson' 'Merrill Nelson'
 'Cindee Beard' 'Mike Noel' 'Ty Markham' 'Rob Bishop' 'Craig Bowden'
 'Chadwick Fairbanks' 'Peter Clemens' 'Sherry Phipps' 'Scott Sandall'
 'Lee Perry' 'Angela Urrea' 'Mike Smith' 'Laurence Kotlikoff'
 'Tony Valdivia' 'L.S. Brown' 'Lyle Hillyard' 'Edward Redd' 'Val Potter'
 'David Clark' 'R Curt Webb' 'Stephen Tryon' 'Jason Chaffetz'
 'David Hinkins' 'Heidi Redd' 'Brad King' 'Christine Watkins'
 'Cole Capener' 'Logan Wilde' 'Fuente' 'Giordani' 'Stephen Park'
 'Hoefling' 'Limbaugh' 'Dustin Baird' 'Andrew Basiago' 'Emidio Soltyisk'
 'Fox' 'Sheila Samm Tittle' 'Buchnan' 'Marshall Schoenke' 'Janet Reid'
 'Burton' 'Herbert' 'Harper' 'Buxton' 'Alan Yorgason' 'Hartwick'
 'Todd Weiler' 'Kelly Miles' 'Amy Morgan' 'Mike Schultz' 'Bob Buckles'
 'Paul Ray' 'Karianne Lisonbee' 'Rich Miller' 'Brad Wilson' 'Steve Handy'
 'Tiffany Kopp' 'Brent Zimmerman' 'Christine Stenquist' 'Stewart Barlow'
 'Timothy Hawkes' 'Kurt Weiland' 'Raymond Ward' 'Becky Edwards'
 'Jon Marsh' 'Wayne Stevens' 'Scott Chew' 'Carl Albrecht' 'William Groff'
 'Chuck Goode' 'Brad Last' 'John Westwood' 'Collin Simonsen' 'Doug Owens'
 'Mia Love' 'Derrin Owens' 'Gray Herbert' 'Deana Froerer'
 'Allen Christensen' 'Giordani/Anderson' 'Stephen Paul Parks'
 'Tom Hoefling' 'Emidio Soltysik' 'Jamin Burton' 'Robert Buchanan'
 'Cherunda Fox' 'David Limbaugh' 'Luz Escamilla' 'Fred Johnson'
 'Wayne Harper' 'Jim Dexter' 'Celina Milner' 'Brian Shiozawa'
 'Ash Anderson' 'Dan Paget' 'Lincoln Fillmore' 'Jake Anderegg'
 'David (Dave) Lifferth' 'Steve Hartwick' 'Susan Duckworth' 'Jamie White'
 'Scott Hawkins' 'Sandra Hollins' 'Rebecca Chavez Houck' 'Joel Briscoe'
 'Angela Romero' 'Brian King' 'Frank Bedolla' 'Mike Winder'
 'Sophia Dicaro' 'Elizabeth Weight' 'Suzanne Harrison' 'Lavar Christensen'
 'Craig Hall' 'Peter Tomala' 'Karen Kwan' 'Macade Jensen'
 'Patricia Phaklides' 'Chelsea Travis' 'Mark Wheatley' 'Patrice Arent'
 'Kris Kimball' 'Carol Spackman' 'Charles Henderson' 'Eric Hutchings'
 'Jim Dunnigan' 'Paul Schulte' 'Lynn Hemingway' 'Joseph Breault'
 'Chad Harrington' 'Dan McCay' 'Kim Coleman' 'Edgar Harwood'
 'Adam Gardiner' 'Christine Passey' 'Bruce Cutler' 'Steve Eliason'
 'Nikki Cunard' 'Marie Poulson' 'Lee Anne Walker' 'Ken Ivory'
 'John Rendell' 'Robert Spendlove' 'Zach Robinson' 'Susan Pulsipher'
 'Patty Rich' 'Greg Hughes' 'Kyle Waters' 'Gordon Jones' 'Garr Smith'
 'John Knotwell' 'Write In' 'Rudi Kohler' 'Tim Quinn' 'Rick Pollock'
 'Douglas Sagers' "'Rocky' De La Fuente" 'Deidre Henderson'
 'Andrew Apsley' 'Dan Hemmert' 'Joe Buchman' 'Curt Crosby' 'Curt Bramble'
 'Jason Christensen' 'Jefferson Moss' 'Aaron Davis' 'Donna Gibbons'
 'Cory Maloy' 'Mike Kennedy' 'Keven Stratton' 'Kay Christofferson'
 'Brian Greene' 'Val Peterson' 'Rachel Nelson' 'Bradley Daw'
 'B. Swallow-Fenton' 'Tommy Williams' 'Keith Grover' 'Dean Sanpei'
 'Nathan Smith Jones' 'Norm Thuston' 'Francis Gibson' 'Mike McKell'
 'Marc Roberts' 'Don Ipson' 'Dorothy Engelman' 'Jon Stanard'
 'V Lowry Snow' 'Walt Brooks' 'Giordani,Rocky' 'Gregg Buxton'
 'Floyd Handley' 'Justin Fawson' 'Matthew Frandsen' 'Gage Froerer'
 'Jeremy Peterson' 'Kathie Darby' 'Dixon Pitcher' 'Jesus Garcia'
 'Derryck Gordon'
}